#Open Context Documentaion Loader

## Setup for Demo/Lab

- Git clone https://github.com/OperationalizingAI/DevOpsGAIWorkshop.git

- Copy *.py and *.txt to root directory

In [ ]:
%pip install --quiet langchain-community langchain unstructured
%pip install --quiet pandas sentence-transformers scikit-learn numpy
%pip install --quiet pymongo langchain-openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.4 MB/s 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.document_loaders import TextLoader

from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

import pandas as pd
import numpy as np
import time
import os

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

from langchain.chains import RetrievalQA
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import (
    DirectoryLoader,
    UnstructuredMarkdownLoader,
)

In [ ]:
!pip install google-cloud-secret-manager
!pip install --upgrade google-auth

from google.cloud import secretmanager
from google.colab import auth
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 2.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.28.2 which is incompatible.
tensorflow 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.


In [ ]:
def load_secrets(secrets_name, project_id):
  # Build a client
  auth.authenticate_user()
  client = secretmanager.SecretManagerServiceClient()
  secret_name = secrets_name
  # Create path to latest secret
  resource_name = f"projects/{project_id}/secrets/{secret_name}/versions/latest"
  # Get your secret :
  response = client.access_secret_version(request={"name": resource_name})
  secret_string = response.payload.data.decode('UTF-8')
  return secret_string

In [ ]:
project_id = 'botchagalupep1'
openai_api_key = load_secrets("openai_api_key",project_id)
os.environ['OPENAI_API_KEY'] = openai_api_key
#MONGODB_ATLAS_CLUSTER_URI = load_secrets("mdb_uri",project_id)
MONGODB_ATLAS_CLUSTER_URI = load_secrets("MDB_CLUSTER0_URI",project_id)
langsmith_api_key = load_secrets("langsmith_api_key",project_id)
#print(langsmith_api_key )
#print(MONGODB_ATLAS_CLUSTER_URI)

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from pymongo import MongoClient

# initialize MongoDB python client
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

DB_NAME = "Cluster0"
COLLECTION_NAME = "OpenContext0"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"

embedded_items_collection = client[DB_NAME][COLLECTION_NAME]

In [ ]:
doc_count = embedded_items_collection.count_documents (filter = {})
print (f"Document count before delete : {doc_count:,}")

result = embedded_items_collection.delete_many(filter= {})
print (f"Deleted docs : {result.deleted_count}")


Document count before delete : 0
Deleted docs : 0


# Load Example 1
## Load Unstructured Markdown (Directory Loader)

In [ ]:
loader = DirectoryLoader(
    "/content/gdrive/MyDrive/GAI/catalog-yaml-format", glob="*.md", loader_cls=UnstructuredMarkdownLoader
)
docs = loader.load()
docs

# Load Example 2
## Load Unstructured Markdown (Directory Loader - With Chunking)


In [ ]:
# load text splitter and split docs into snippets of text
text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)
docs = split_docs

# Load Example 3
## Load Unstructured Markdown (File Loader)


In [ ]:
# Example usage
directory_path = '/content/gdrive/MyDrive/GAI/catalog-yaml-format'
markdown_contents = []

for file in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, file)):
        full_path = os.path.join(directory_path, file)
        loader = UnstructuredMarkdownLoader(full_path)
        #loader = UnstructuredMarkdownLoader(full_path)
        md = loader.load()
        markdown_contents.extend(md)

docs = markdown_contents
print(docs)


[Document(page_content="sidebar_position: 3\n\nEntity Reference\n\nEntities commonly have a need to reference other entities. For example, a\xa0CodeComponent\xa0entity may want to declare who its owner is by mentioning a Team or Person entity, and a Person entity may want to declare what Team entities it is a member of. This describes how to write those references in your yaml entity declaration files.\n\nEach entity in OpenContext is uniquely identified by the triplet of its\xa0kind,\xa0namespace, and\xa0name. But that's a lot to type out manually, and in a lot of circumstances, both the kind and the namespace are fixed, or possible to deduce, or could have sane default values.\n\nEach reference can be expressed in one of two ways: as a compact string, or as a compound reference structure.\n\nString References\n\nThis is the most common alternative, that should be used in almost all circumstances.\n\nThe string is on the form\xa0[<kind>:][<namespace>/]<name>, that is, it is composed o

# Load Example 4
##Load Unstructured Markdown (File Loader - elements mode)


In [ ]:
# Example usage
directory_path = '/content/gdrive/MyDrive/GAI/catalog-yaml-format'

for file in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, file)):
        full_path = os.path.join(directory_path, file)
        loader = UnstructuredMarkdownLoader(full_path,mode="elements")
        #loader = UnstructuredMarkdownLoader(full_path)
        md = loader.load()
        markdown_contents.extend(md)

docs = markdown_contents
print(docs)


[Document(page_content="sidebar_position: 3\n\nEntity Reference\n\nEntities commonly have a need to reference other entities. For example, a\xa0CodeComponent\xa0entity may want to declare who its owner is by mentioning a Team or Person entity, and a Person entity may want to declare what Team entities it is a member of. This describes how to write those references in your yaml entity declaration files.\n\nEach entity in OpenContext is uniquely identified by the triplet of its\xa0kind,\xa0namespace, and\xa0name. But that's a lot to type out manually, and in a lot of circumstances, both the kind and the namespace are fixed, or possible to deduce, or could have sane default values.\n\nEach reference can be expressed in one of two ways: as a compact string, or as a compound reference structure.\n\nString References\n\nThis is the most common alternative, that should be used in almost all circumstances.\n\nThe string is on the form\xa0[<kind>:][<namespace>/]<name>, that is, it is composed o

# Load Example 5
## Load MarkdownHeaderTextSplitter



In [ ]:
def read_markdown_files(directory_path):
    """
    Reads and returns the content of all Markdown files in the given directory path.

    :param directory_path: Path to the directory whose Markdown files are to be read.
    :return: A list of dictionaries, each containing the file name and content of a Markdown file.
    """
    markdown_contents = []

    # Check if the given path is a directory
    if not os.path.isdir(directory_path):
        print(f"The path {directory_path} is not a valid directory.")
        return markdown_contents

    # Find all Markdown files in the directory
    markdown_files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f)) and f.endswith('.md')]

    for file in markdown_files:
        file_path = os.path.join(directory_path, file)
        with open(file_path, 'r', encoding='utf-8') as md_file:
            content = md_file.read()

            #print(file)
            markdown_splitter = MarkdownHeaderTextSplitter(
                 headers_to_split_on=headers_to_split_on, strip_headers=False)
            md = markdown_splitter.split_text(str(content))

            docx = []
            for mdx in md:
              mdx.metadata['source'] = file
              docx.append(mdx)

            #print(docx)
            markdown_contents.extend(docx)

    return markdown_contents

Need to add source document in the meta data

In [ ]:
# Example usage


headers_to_split_on = [
                       ("#", "Header 1"),
                       ("##", "Header 2"),
                       ("###", "Header 3"),
                       ("####", "Header 4"),
                       ("#####", "Header 5"),
                       ('\n\n<Tab name="',"Tab Name"),
                       ("\n\n<Tabs>\n\n","Tabs Container"),
                       ("<table>\n","Table"),
                       ("<tr>\n","Table Row"),
                       ("<th>\n","Header Cell"),
                       ("<td>\n","Data Cell"),
                       #("\n```","Code Block"),
                       ("\n\n***\n\n","Horizontal Rule"),
                       ("\n\n---\n\n","Horizontal Rule"),
                       ("\n\n","Whitespace"),
                       ("\n\n\n","Whitespace"),
                       ("\n","Newline"),
                       (" ","Space"),
                       #("","Empty String"),
                       ("`","Backtick")

                      # From packages/mongodb-rag-ingest/src/embed/chunkMd.ts
                    ]
directory_path = '/content/gdrive/MyDrive/GAI/catalog-yaml-format'
docs = read_markdown_files(directory_path)

print("Number of docs:", len(docs))
for doc in docs:
  print(len(doc.page_content))
  print(doc)
#print(docs

Number of docs: 75
27
page_content='---\nsidebar_position: 3\n---' metadata={'source': 'entity-reference.md'}
748
page_content="# Entity Reference  \nEntities commonly have a need to reference other entities. For example, a\xa0CodeComponent\xa0entity may want to declare who its owner is by mentioning a Team or Person entity, and a Person entity may want to declare what Team entities it is a member of. This describes how to write those references in your yaml entity declaration files.  \nEach entity in OpenContext is uniquely identified by the triplet of its\xa0[kind,\xa0namespace, and\xa0name](common). But that's a lot to type out manually, and in a lot of circumstances, both the kind and the namespace are fixed, or possible to deduce, or could have sane default values.  \nEach reference can be expressed in one of two ways: as a compact string, or as a compound reference structure." metadata={'Header 1': 'Entity Reference', 'source': 'entity-reference.md'}
1817
page_content="## **Strin

In [ ]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings

embedding = "text-embedding-3-small"

# insert the documents in MongoDB Atlas with their embedding
vector_search = MongoDBAtlasVectorSearch.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(model="text-embedding-3-small",dimensions=1536,disallowed_special=()),
    collection=embedded_items_collection,
    index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,
)